In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model

import tensorflow as tf

import pandas as pd

import os
import pickle
import numpy as np

import scipy.sparse as sp
import scipy.io as spio

import isolearn.io as isoio
import isolearn.keras as iso


Using TensorFlow backend.


In [2]:
#Load sequence data

df = pd.read_csv('../../../aparent/data/polyadb_features_pas_3_utr3_isoforms.csv', sep='\t')

save_dict = np.load("../../../aparent/data/polyadb_features_pas_3_utr3_isoforms_no_x.npz")
m, l = save_dict['m'], save_dict['l']


/home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (200,201,203,207,208,210) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Load APARENT Resnet

model_name = 'aparent_all_libs_resnet_no_clinvar_wt_ep_5'

save_dir = os.path.join(os.getcwd(), '../../../aparent-resnet/saved_models')
model_path = os.path.join(save_dir, model_name + '.h5')

aparent_model = load_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
#Score all sequences with APARENT (use sum of cuts to capture OR-like logic)

max_n_pas = 30

encoder = iso.OneHotEncoder(205)

a = np.zeros((len(df), max_n_pas))
a_all_cuts = np.zeros((len(df), max_n_pas))

for k in range(max_n_pas) :
    
    print("Predicting for PAS #" + str(k) + "...")
    
    df.loc[df['wide_seq_ext_' + str(k)].isnull(), 'wide_seq_ext_' + str(k)] = 'X' * 205
    
    onehots = np.concatenate([encoder.encode(row['wide_seq_ext_' + str(k)][175-70:175-70+205])[None, None, :, :] for _, row in df.iterrows()], axis=0)
    
    fake_lib = np.zeros((onehots.shape[0], 13))
    fake_lib[:, 11] = 1.
    
    #Pad
    n_pad = 32 - onehots.shape[0] % 32 if onehots.shape[0] % 32 != 0 else 0

    fake_lib = np.concatenate([fake_lib, np.zeros((n_pad, 13))], axis=0)
    onehots = np.concatenate([onehots, np.zeros((n_pad, 1, 205, 4))], axis=0)
    
    _, pred_cuts = aparent_model.predict(x=[onehots, fake_lib], batch_size=32, verbose=1)
    
    if n_pad > 0 :
        pred_cuts = pred_cuts[:-n_pad, :]
    
    isoform_start = 77
    isoform_end = 127

    pred_iso = np.sum(pred_cuts[:, isoform_start:isoform_end], axis=1)
    score = np.log(pred_iso / (1. - pred_iso))

    isoform_start = 0
    isoform_end = 205

    pred_iso_all_cuts = np.sum(pred_cuts[:, isoform_start:isoform_end], axis=1)
    score_all_cuts = np.log(pred_iso_all_cuts / (1. - pred_iso_all_cuts))

    a[:, k] = score[:]
    a_all_cuts[:, k] = score_all_cuts[:]

a = a * m
a = np.clip(a, -10., 10.)

a_all_cuts = a_all_cuts * m
a_all_cuts = np.clip(a_all_cuts, -10., 10.)


Predicting for PAS #0...
10432/10432 [==============================] - 85s 8ms/step
Predicting for PAS #1...
10432/10432 [==============================] - 105s 10ms/step
Predicting for PAS #2...
10432/10432 [==============================] - 77s 7ms/step
Predicting for PAS #3...
10432/10432 [==============================] - 79s 8ms/step
Predicting for PAS #4...
10432/10432 [==============================] - 74s 7ms/step
Predicting for PAS #5...
10432/10432 [==============================] - 73s 7ms/step
Predicting for PAS #6...
10432/10432 [==============================] - 73s 7ms/step
Predicting for PAS #7...
10432/10432 [==============================] - 73s 7ms/step
Predicting for PAS #8...
10432/10432 [==============================] - 75s 7ms/step
Predicting for PAS #9...
10432/10432 [==============================] - 75s 7ms/step
Predicting for PAS #10...
10432/10432 [==============================] - 72s 7ms/step
Predicting for PAS #11...
10432/10432 [=======================

In [5]:
#Dump prediction matrix

np.save('apa_polyadb_data/' + model_name + '_native_scores_utr3_isoforms', a)
np.save('apa_polyadb_data/' + model_name + '_native_scores_utr3_isoform_all_cuts', a_all_cuts)
